In [9]:
import numpy as np
import pandas as pd
import os
import sys
import random

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from Database.database_manager import DatabaseManager
from utils.Fighter_Style import get_all_fighter_styles

In [2]:
with DatabaseManager('../../../Database/fighters.db') as db:
    fights = db.get_fights()
    fighter_ids = db.get_fighter_ids()

In [3]:
fights = pd.DataFrame(fights)
fights.head()

,fight_id,red_fighter_id,blue_fighter_id,winner_id,event_date,win_method,final_round,red_knockdowns,red_sig_strikes,red_takedowns,...,blue_takedowns,is_completed,event_url,red_sub_attempts,blue_sub_attempts,final_time_seconds,red_fighter_elo_before,blue_fighter_elo_before,red_fighter_elo_after,blue_fighter_elo_after
0,1,2899,3774,NaN,2021-03-13,CNC,2,0,19,0,...,0,1,http://ufcstats.com/event-details/8c90c1563972...,0,0,18,1626.759208,1630.968948,1626.895862,1630.833430
1,2,1483,2126,1483.0,2021-03-13,KO/TKO\n\n \n\n Punches,1,1,16,0,...,0,1,http://ufcstats.com/event-details/8c90c1563972...,0,0,71,1599.310991,1529.172145,1609.069693,1518.465845
2,3,4115,1119,4115.0,2021-03-13,KO/TKO\n\n \n\n Punch,1,1,1,0,...,0,1,http://ufcstats.com/event-details/8c90c1563972...,0,0,22,1627.490218,1406.265316,1632.474205,1397.739950
3,4,607,4274,607.0,2021-03-13,KO/TKO\n\n \n\n Punch,2,1,48,0,...,0,1,http://ufcstats.com/event-details/8c90c1563972...,0,0,183,1559.895877,1584.538271,1568.841704,1576.059124
4,5,2665,3638,2665.0,2021-03-13,S-DEC,3,0,55,2,...,0,1,http://ufcstats.com/event-details/8c90c1563972...,1,0,300,1479.758222,1686.221243,1485.432011,1679.282146


In [4]:
fighter_styles = get_all_fighter_styles(fighter_ids)
styles_df = pd.DataFrame(fighter_styles)

In [5]:
styles_df.head(200)

,fighter_id,primary_style,secondary_style,tertiary_attributes
0,2107,Newcomer,Newcomer,Newcomer
1,1709,Newcomer,Newcomer,Newcomer
2,1541,Power Grappler,Grinding Decision Fighter,Conventional Frame (Orthodox)
3,2402,Pure Striker,Counter Decision Fighter,Conventional Frame (Switch)
4,3761,Wrestle-Boxer,Grinding Finisher,Conventional Frame (Orthodox)
...,...,...,...,...
195,4004,Wrestle-Boxer,Grinding Decision Fighter,Conventional Frame (Orthodox)
196,797,Newcomer,Newcomer,Newcomer
197,2692,Pure Striker,Pressure Decision Fighter,Conventional Frame (Orthodox)
198,1245,Power Grappler,Grinding Decision Fighter,Conventional Frame (Orthodox)


In [17]:
styles_lookup = styles_df.set_index('fighter_id')

model_data = []

for _, fight in fights.iterrows():
    red_id = fight['red_fighter_id']
    blue_id = fight['blue_fighter_id']
    winner_id = fight['winner_id']

    try:
        red_style = styles_lookup.loc[red_id]
        blue_style = styles_lookup.loc[blue_id]
    except KeyError:
        continue

    fighters = [
        {'id' :red_id, 'style':red_style},
        {'id' :blue_id, 'style':blue_style}
    ]

    random.shuffle(fighters)

    fighter_1 = fighters[0]
    fighter_2 = fighters[1]

    if fighter_1['id'] == winner_id:
        fighter_1_win = 1
    elif fighter_2['id'] == winner_id:
        fighter_1_win = 0
    else:
        fighter_1_win = 0.5

    new_row = {
        'primary_1': fighter_1['style']['primary_style'],
        'secondary_1': fighter_1['style']['secondary_style'],
        'tertiary_1': fighter_1['style']['tertiary_attributes'],

        'primary_2': fighter_2['style']['primary_style'],
        'secondary_2': fighter_2['style']['secondary_style'],
        'tertiary_2': fighter_2['style']['tertiary_attributes'],

        'fighter_1_win': fighter_1_win
    }
    model_data.append(new_row)

model_data = pd.DataFrame(model_data)
model_data.head(51)

,primary_1,secondary_1,tertiary_1,primary_2,secondary_2,tertiary_2,fighter_1_win
0,Power Grappler,Grinding Decision Fighter,Conventional Frame (Orthodox),Wrestle-Boxer,Grinding Decision Fighter,Conventional Frame (Southpaw),0.5
1,Wrestle-Boxer,Grinding Finisher,Conventional Frame (Orthodox),Wrestle-Boxer,Grinding Finisher,Conventional Frame (Orthodox),0.0
2,Power Grappler,Grinding Decision Fighter,Conventional Frame (Southpaw),Striker,Paced Decision Fighter,Conventional Frame (Orthodox),0.0
3,Striker,Paced Power Puncher,Conventional Frame (Southpaw),Wrestle-Boxer,Grinding Decision Fighter,Conventional Frame (Orthodox),0.0
4,Wrestle-Boxer,Grinding Power Puncher,Conventional Frame (Orthodox),Striker,Pressure Power Puncher,Conventional Frame (Southpaw),1.0
5,Wrestle-Boxer,Grinding Decision Fighter,Conventional Frame (Orthodox),Wrestle-Boxer,Grinding Decision Fighter,Conventional Frame (Southpaw),0.5
6,Wrestle-Boxer,Grinding Decision Fighter,Conventional Frame (Southpaw),Wrestle-Boxer,Grinding Decision Fighter,Conventional Frame (Orthodox),0.0
7,Striker,Pressure Finisher,Conventional Frame (Switch),Striker,Pressure Decision Fighter,Conventional Frame (Orthodox),1.0
8,Power Grappler,Grinding Decision Fighter,Conventional Frame (Orthodox),Striker,Counter Decision Fighter,Conventional Frame (Orthodox),1.0
9,Striker,Pressure Decision Fighter,Conventional Frame (Southpaw),Power Grappler,Grinding Decision Fighter,Conventional Frame (Orthodox),1.0
